In [7]:
#-------------------------------------------------#
#                                                 #
#  realiser par: hajji youssef & mohamed khachchab#
#  Nom fichier : VF                            #
#  date de depot: 30/11/2022                      #   
#                                                 #
#-------------------------------------------------#
#les import
import tkinter as tk
from tkinter import *
import json
import random as rd
import time
#---------------------------------------------------------Class Space Invader---------------------------------#
class SpaceInvaders(object):
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Space Invaders")
        self.root.resizable(0, 0)
        self.frame = tk.Frame(self.root)
        self.frame.pack(side="top", fill="both")
        self.game = Game(self.frame)
            
    def play(self):
        #saissie du nom du joueuer et lancement du jeu
        def debut(event):
            self.input_name.config(state='disabled') #desactiver le champs de saisie
            self.game.score.set_name(self.input_name.get()) #ajouter du joueur au class Score()
            self.root.bind("<Key>", self.game.Click) #deplacement et tire du defender
            self.game.start_animation() #start d'annimation     
        nom = Label(self.frame, text="Enter votre Nom : ").pack(side = TOP) #message d'affichage
        self.input_name = Entry(self.frame,bd =8)
        self.input_name.pack() #pack() organise les widgets dans des boîtes horizontales et verticales
        self.input_name.focus_set() #méthode utilisée pour définir le focus sur le widget
        self.input_name.bind('<Return>',debut) #on cliquant sur entrer on appelle la fonction debut   
        self.root.mainloop()
#----------------------------------------Class GAME --------------------------------------------------------------------#
class Game(object):
    def __init__(self, frame):
        self.frame = frame#construction du frame
        self.fleet = Fleet()#appel de la classe fleet dans game
        self.score = Score()#appel de la classe score dans game
        self.defender = Defender()#appel de la classe defender dans game
        self.bunckers= LesBunckers()#appel de la classe lesbunkers dans game
        
    #le canvas
        self.canvas_height = self.fleet.get_height() #initialisé dans la classe Fleet
        self.canvas_width = self.fleet.get_width()
        self.canvas = tk.Canvas(self.frame,width=self.canvas_width,height=self.canvas_height,bg='black')
        self.canvas.pack(padx=5,pady=5,side ="bottom")
    #les install_in
        self.defender.install_in(self.canvas)#installation du defender
        self.fleet.install_in(self.canvas)#installation de la flotte d'aliens
        self.bunckers.install_in(self.canvas)#installation du bunckers
        #les accesseur
    def get_canvas_height(self):
        return self.canvas_height
    def get_canvas_width(self):
        return self.canvas_width
    
    #gestion des touches
    def Click(self, event):
        cond_x = self.defender.xa  #pour limiter le cadre du jeu
        if event.keysym == 'Left':
            if cond_x - self.defender.move_delta > 0:#pour que le defender ne sort pas du cadre
                self.defender.xa = self.defender.xa - self.defender.move_delta#modification du x
                self.defender.move_in(self.canvas,-self.defender.move_delta)                
        elif event.keysym == 'Right':
            if cond_x + self.defender.move_delta < self.defender.canvas_width:   #pour que le defender ne sort pas du cadre
                self.defender.xa = self.defender.xa + self.defender.move_delta    #changement de la valeur du x
                self.defender.move_in(self.canvas,self.defender.move_delta)
        elif event.keysym == 'space':  #button pour tirer
            if len(self.fleet.aliens_fleet) != 0 : #verification si le jeu a fini ou pas 
                self.defender.feu(self.canvas)
    #affichage de score
    def show_score(self):
        self.score_actuel = StringVar()  #gérer la valeur du score actuel dans la widget 
        score_label = Label(self.canvas, textvariable = self.score_actuel, bg="green", fg="white", font="Arial 15")#affichage du score
        self.score_actuel.set("score : " + str(self.score.get_points())) #modification du score 
        self.canvas.create_window(1050, 25, window = score_label)
    #l'enregistrement du score à la fin du jeu
    def saveFile(self):
        self.score.delai = int(time.time()) - self.score.start_time #delai:durée de jeu
        self.score.toFile("LastScore.json") #enregistrement du dernier score
        lastScore = self.score.fromFile("LastScore.json") #recuperation du dernier score
        self.resultats = Resultats.fromFile("AllResults.json")
        self.resultats.ajout(lastScore) #l'ajout du dernier score au fichier resultat
        self.resultats.toFile("AllResults.json") #enregistrement de tous les resultats de tous les parties
    #si le joueur perd
    def joueur_perd(self):
        self.score.winning = False #le joueur a perdu
        self.saveFile() 
        w = int(self.canvas.cget("width"))//2 #positionnement du message
        h =int(self.canvas.cget("height"))//2
        self.canvas.delete("all") #nettoyage du canvas
        self.canvas.create_text(w,h,text="Game Over", fill="red", font="Arial 15")# style d'affichage du message
    
    #si le joueur a gagné
    def joueur_gagné(self):
        self.score.winning = True #le joueur a gagné
        self.saveFile()
        w= int(self.canvas.cget("width"))//2 #positionnement du message
        h = int(self.canvas.cget("height"))//2
        self.canvas.delete("all")#nettoyage du canvas
        self.canvas.create_text(w,h,text=" winner winner chiken dinner !! ", fill="red", font="Arial 15")# style d'affichage du message
    #debut d'animation
    def start_animation(self):
        self.show_score() #affichage du score
        self.fleet.previous_shoot_time = float(time.time()) #l'ecart entre chaque tire 
        self.animation() #appelle de l'animation
    def animation(self):
        self.score_actuel.set("score : "+str(self.score.get_points())) #actualisation du score
        if len(self.fleet.aliens_fleet)!= 0: #il y a encore des aliens
            x1,y1,x2,y2=self.canvas.bbox("alien")#retourner un tuple de coordonnees(x1,y1,x2,y2)
            #fleet en dessus du defender 
            if y2<=int(self.canvas.cget("height"))-self.defender.height:
                self.defender.deplacement_bullet(self.canvas) 
                self.fleet.move_in(self.canvas)
                self.fleet.touched_aliens(self.canvas,self.defender,self.score)
                self.bunckers.buncker_touched(self.canvas,self.fleet)
                self.canvas.after(50,self.animation) #appelle a la fonction apres 50ms
            else: #cas de perte
                self.joueur_perd()
        else: #cas de gain
            self.joueur_gagné()      
#---------------------------------------------------Class DEFENDER--------------------------------------------#
class Defender(object):
    def __init__(self):
        #images utilisés
        self.defender_img = PhotoImage(file="defender.gif")
        self.width = self.defender_img.width()
        self.height = self.defender_img.height()
        #la taille du canvas
        self.canvas_height = Fleet().get_height()
        self.canvas_width = Fleet().get_width()
        #initialisation des varibales
        self.id = None
        self.xa = self.canvas_width//2 - self.width#positionement du defender
        self.yb = self.canvas_height - self.height
        self.move_delta = self.width/2 #deplacement horizontale
        #tire des balles
        self.max_fired_bullets = 8
        self.fired_bullets = []
    #les accesseurs
    def get_height(self):
        return self.height
    def get_width(self):
        return self.width
    #insertion de l'image du defender
    def install_in(self, canvas):
        self.id = canvas.create_image(self.xa ,self.yb, image = self.defender_img , tags="defender")            
    #deplacement horizontal du defender
    def move_in(self , canvas , dx):   
        canvas.move(self.id, dx, 0)
    #fonction de tire du defender
    def feu(self, canvas):
        if len(self.fired_bullets) < self.max_fired_bullets: #non depassement du nbr du max du bullets
            bullet = Bullet("defender")#bullets sortant du defender
            bullet = bullet.install_in(canvas) #affichage des bullets 
            self.fired_bullets.append(bullet) #l'ajout des bullets tirers à la liste fired_bullets
    #deplacement verticale des balles
    def deplacement_bullet(self,canvas):
        for i in range(0,len(self.fired_bullets)):
            x1,y1,x2,y2 = canvas.bbox(self.fired_bullets[i].id) #coordonnées de chaque balle en tuple
            if y1<0: #si le bullet depasse le cadre du jeu 
                canvas.delete(self.fired_bullets[i].id) #suprression du dessin du bullet
                del self.fired_bullets[i] #delete de sa valeur du liste
                break
            else:
                self.fired_bullets[i].move_in(canvas)   #deplacement du bullet               
#-----------------------------------------------------Class BULLET----------------------------------------------#
class Bullet(object):
    #méthode init
    def __init__(self, shooter):
        self.shooter = shooter
        self.radius = 15 #rayon bullets
        self.color = "green"#couleur bullets
        self.speed = 10#vitesse du bullets
    #affichage du bullet du defender
    def install_in(self, canvas):
        if self.shooter=="defender":
            w = Defender().get_width() 
            h = Defender().get_height() #recuperation de la taille du defender    
            x,y = canvas.coords("defender") #recuperation des coordonnées 
            x,y = x - w/4 , y - h #positionnement au dessus du defender
            ra = self.radius
            self.id = canvas.create_oval(x, y, x+ra, y+ra, fill = self.color)
            return self
    #le mvt verticale des bullets
    def move_in(self,canvas):
        if self.shooter=="defender":
            canvas.move(self.id, 0, -self.speed) #vers le haut pour le defender         
#-----------------------------------------------------Class ALIEN-------------------------------------------------#
class Alien(object):
    def __init__(self):
        self.alive = True
        self.alien = PhotoImage(file="alien.gif") #image d'alien
        self.boom = PhotoImage(file="bunker.gif")#image du bunker
        self.alien_width = self.alien.width()
        self.alien_height = self.alien.height()    
    def get_width(self):
        return self.alien_width
    def get_height(self):
        return self.alien_height
    #affichage de l'alien
    def install_in(self, canvas, x, y): 
        self.id = canvas.create_image(x, y, image=self.alien, tags="alien")
        return self
    #déplacement de l'alien
    def move_in(self, canvas, dx, dy):
        canvas.move(self.id, dx, dy)
    #si l'alien est touché par une bullets
    def touched(self,canvas,project):
        canvas.delete(project.id)  #suprression du bullet
        canvas.delete(self.id)    #supression de l'alien
        self.alive = False  #l'alien est mort  
       
 #--------------------------------------Class FLEET------------------------------------------------#
class Fleet(object):
    def __init__(self):
        #taille du canvas
        self.width = 1200 #largeur 
        self.height = 600 #hauteur 
        #taille du fleet
        self.aliens_lines = 4
        self.aliens_columns = 6
        self.aliens_inner_gap = 20#espace entre les aliens
        fleet_size = self.aliens_lines * self.aliens_columns
        self.aliens_fleet = [None] * fleet_size
        #vitesse du deplacement du fleet
        self.alien_x_delta = 10
        self.alien_y_delta = 18
        #animation 
        self.animation_start_time = float(time.time())
    #les accesseurs 
    def get_width(self):
        return self.width
    def get_height(self):
        return self.height
    #installation du fleet
    def install_in(self, canvas):
        x, y = 50 , 75   #coordonnées du position initial
        position = 0 #l'indice dans la liste
        
        for i in range(0,self.aliens_lines):
            for j in range(0,self.aliens_columns):
                alien = Alien()
                self.aliens_fleet[position] = alien.install_in(canvas,x,y) 
                position=position+1
                x=x+ self.aliens_inner_gap + alien.alien_width
            x = 50
            y=y+ self.aliens_inner_gap + alien.alien_height  
    #deplacement du fleet
    def move_in(self, canvas): 
        if len(self.aliens_fleet) != 0:
            x1,y1,x2,y2 = canvas.bbox("alien")
            if x2 >= int(canvas.cget("width")): 
                self.alien_x_delta = -self.alien_x_delta
                dy = self.alien_y_delta
            elif x1 <= 0: 
                self.alien_x_delta = -self.alien_x_delta
                dy = self.alien_y_delta
            else:
                dy = 0
            for i in range(0,len(self.aliens_fleet)):
                self.aliens_fleet[i].move_in(canvas,self.alien_x_delta,dy)
    
    #cas de collision entre alien et balles de defender
    def touched_aliens(self,canvas,defender,score):
        sor1 = False #condition d'arreter boucle de fleet
        sor2 = False #condition d'arreter boucle de bullets
        for i in range(len(self.aliens_fleet)): 
            x1,y1,x2,y2 = canvas.bbox(self.aliens_fleet[i].id) #coordonnées de chaque alien sous forme de tuple
            overlapped = canvas.find_overlapping(x1, y1, x2, y2)#revoie les elementes du rectangle 
            if len(overlapped) > 1 : #il y a plus d'un seul element
                for j in range(len(defender.fired_bullets)):
                    for k in range(len(overlapped)):
                        if defender.fired_bullets[j].id == overlapped[k]: #l'indice j contient le self du l'alien 
                            self.expo_boom(canvas,defender.fired_bullets[j]) #collision
                            self.aliens_fleet[i].touched(canvas,defender.fired_bullets[j]) #suppression du canvas
                            score.refresh_score(time.time()) #augumentation du score
                            del defender.fired_bullets[j]   #supprimer de valeur du liste
                            del self.aliens_fleet[i]
                            sor1 = True
                            sor2 = True
                            break; #fin de la boucle overlapped
                    if sor2 == True: #fin de la boucle de fired_bullets
                        break;
                if sor1 == True: #fin de la boucle aliens_fleet
                    break;    

    #si l'alien est touché par une balle
    def expo_boom(self,canvas,project):
        self.boom = PhotoImage(file="explosion.gif")
        x1,y1,x2,y2 = canvas.bbox(project.id) #recuperation du coord du project
        x,y = x1+(x2-x1)/2, y1+(y2-y1)/2
        boom = canvas.create_image(x, y, image=self.boom, tags="boom")
        canvas.after(50,canvas.delete,boom) #suppression de l'effet de Boom

#--------------------------------------------------class Les Bunckers--------------------------------------------------#
class LesBunckers(object):
    def __init__(self):
        self.nb_bunckers = 6
        self.bunckers_liste = [None] * self.nb_bunckers
        self.dx = Fleet().get_width()/10 #entre chaque buncker
    

    #l'installation des bunckers
    def install_in(self,canvas):
        xa = int(canvas.cget("width"))/5 #recuperation de la largeur
        for i in range(0,self.nb_bunckers):
            buncker = Buncker()
            yb = int(canvas.cget("height")) - 2*Defender().get_height() - buncker.buncker_height #recuperation de la largeure
            self.bunckers_liste[i] = buncker.install_in(canvas, xa, yb) #return le self, install pour chaque buncker
            xa =xa+ self.dx     
    #la collision entre bunkcer est alien
    def buncker_touched(self, canvas, fleet):
        sor1 = False
        sor2 = False
        for i in range(len(self.bunckers_liste)):
            x1, y1, x2, y2 = canvas.bbox(self.bunckers_liste[i].id) #coordonnées du buncker
            overlapped = canvas.find_overlapping(x1, y1, x2, y2)
            if len(overlapped) > 1:
                for j in range(len(fleet.aliens_fleet)):
                    for k in range(len(overlapped)):
                        if fleet.aliens_fleet[j].id == overlapped[k]: 
                            self.touched(canvas, fleet.aliens_fleet[j]) #effet de boom
                            canvas.delete(self.bunckers_liste[i].id) #supression des banckers
                            canvas.delete(fleet.aliens_fleet[j].id) #supression des aliens
                            self.bunckers_liste[i].health = 0
                            del self.bunckers_liste[i] #supression bancker from liste
                            del fleet.aliens_fleet[j]
                            sor1 = True
                            sor2 = True
                            break;
                    if sor1 == True:
                        break;
                if sor2 == True:
                    break;
    #pour la collison du buncker avec l'alien
    def touched(self,canvas, alien):
        self.boom = PhotoImage(file="explosion.gif")#image d'explosion
        a,b = canvas.coords(alien.id)
        boom = canvas.create_image(a, b, image=self.boom, tags="boom")
        canvas.after(50,canvas.delete,boom)

#--------------------------------------------------Class Buncker--------------------------------------------------#
class Buncker(object):
    def __init__(self):
        self.id = None
        #images
        self.buncker_img = PhotoImage(file="bunker.gif")
        self.buncker_width = self.buncker_img.width()#recuperation de longeure
        self.buncker_height = self.buncker_img.height()#recuperation de la largeure
    #affichage d'un buncker
    def install_in(self,canvas, xa, yb):
        self.id = canvas.create_image(xa, yb, image = self.buncker_img , tags="bunckers")
        return self  
#--------------------------------------------------Class SCORE-------------------------------------------------#
class Score(object):
    def __init__(self):
        self.points = 0
        self.name = None
        self.winning = False
        self.delai = None
        self.start_time = int(time.time())
        self.delai_recommended = 2*60
    
    #les accesseurs 
    def get_points(self):
        return self.points
    def set_name(self,Nname):
        self.name = Nname
    #changement du score selon le temps
    def refresh_score(self, end_time):
        temps = int(end_time)
        if self.delai_recommended - (temps - self.start_time) > 0:
            self.points += self.delai_recommended - (temps - self.start_time)
        else: #sinon, on ajout 10 pts / kill
            self.points += 10
    def toFile(self,file):
        f = open(file,"w")
        json.dump(self.__dict__,f)
        f.close();
    @classmethod
    def fromFile(cls, file):
        try: #cas normal
            f = open(file,"r")
        except FileNotFoundError:  
            print("Fichier introuvable")
            snew=Score() 
        else:
            d = json.load(f) 
            snew = Score()
            snew.name = d["name"]
            snew.points = d["points"]
            snew.delai = d["delai"]
            snew.winning = d["winning"]
            f.close();
        finally:
            return snew  
    
    #affichage du print()
    def __str__(self):
        return " -> "+ str(self.points)
#----------------------------------------------------Class RESULTAT------------------------------#
class Resultats(object):
    def __init__(self):
        self.lesResultats=[]
    #les accesseurs
    def get_resultats(self):
        return self.lesResultats
    def ajout(self,score):
        self.lesResultats.append(score)
    #affichage du print()
    def __str__(self):
        try:
            ch=str(self.lesResultats[0])
            for elt in self.lesResultats[1:]:
                ch=ch +" , "+ str(elt)
            return ch
        except IndexError: 
            return "Pas de score enregistré"
    
    #enregistrement dans le fichier
    def toFile(self,file):
        f = open(file,"w")
        tmp=[]
        for s in self.lesResultats:
            dic = {}
            
            dic["points"] = s.points
            dic["delai"] = s.delai
            dic["winning"] = s.winning
            tmp.append(dic)
        json.dump(tmp,f)
        f.close();
    
    #recuperation des donnes du fichier
    @classmethod
    def fromFile(cls,file):
        try:
            f = open(file, "r")
        except FileNotFoundError: #si le fichier n'existe pas
            print("Ce fichier n'existe pas")
            res=Resultats()
            res.lesResultats=[]
        else:
            tmp = json.load(f)
            liste = []
            for d in tmp:
                sc=Score()
                
                sc.points = d["points"]
                sc.delai = d["delai"]
                sc.winning = d["winning"]
                liste.append(sc)
            res = Resultats()
            res.lesResultats = liste
            f.close();
        finally:
            return res  
#--------------------Lancer le jeu----------------#
jeu=SpaceInvaders()
jeu.play()